In [3]:
import cv2
import numpy as np
import os
import json

In [4]:
sift = cv2.xfeatures2d.SIFT_create()

In [5]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

In [6]:
uncropped_kpd_cache = {}
print("Building cache...")
files = os.listdir("./prod/sources")
for i, file in enumerate(files):
    print(f"Computing {file} ({i + 1} / {len(files)})")
    uncropped = cv2.imread(f"./prod/sources/{file}")
    if uncropped is None:
        print(f"Error reading {file}. Skipping.")
        continue
    uncropped_keypoints, uncropped_descriptors = sift.detectAndCompute(uncropped, None)
    uncropped_kpd_cache[file] = (uncropped_keypoints, uncropped_descriptors)

Building cache...
Computing 00c07afb2992350db7ee1ea436453331.gif (1 / 500)
Error reading 00c07afb2992350db7ee1ea436453331.gif. Skipping.
Computing 0136d55681ec78cf876b1349e4072205.png (2 / 500)
Computing 014ebb7650ce0021d082ca98822e6dde.jpg (3 / 500)
Computing 021a47ffbe28eb4e202eea7c09a16e70.png (4 / 500)
Computing 025560ad8d201d5c78be65bc5a74d218.jpg (5 / 500)
Computing 0278cb557184d89927b52f2eb6f46e24.png (6 / 500)
Computing 029214ea2b51c637cf55b4ce92b75362.jpg (7 / 500)
Computing 0312941c2310c22755b4cad456bf8b99.png (8 / 500)
Computing 03298bc06992753f116f366a4ddbb0a0.jpg (9 / 500)
Computing 03bafaa27f59efeb9dd7576a69e289b6.jpg (10 / 500)
Computing 04c18d2aca49fb6e1d2b7b963665a439.jpg (11 / 500)
Computing 063a579cc3880b22efcc426e59ce83e9.jpg (12 / 500)
Computing 06d97ce96605716e31a85c3b34a48e50.png (13 / 500)
Computing 06faebda093f1401c4e322a63996d05f.jpg (14 / 500)
Computing 075660fae52c3c0b8ea28edaeb88145d.png (15 / 500)
Computing 076da525042f6e676aae753e6e962b40.png (16 / 500)
C

In [7]:
from keypoint_serialization import *

In [11]:
def serialize_kpd(i):
    (kps, d) = i
    skps = [serialize_kp(kp) for kp in kps]
    return (skps, d.tolist())

In [12]:
for filename, kpd in uncropped_kpd_cache.items():
    subdir = "./prod/kpd_cache/" + filename[0:2]
    if not os.path.isdir(subdir):
        os.mkdir(subdir)
    with open(subdir + "/" + filename + ".json", "w") as f:
        json.dump(serialize_kpd(kpd), f)